In [23]:
import paramiko
from datetime import datetime

def start_index(source):
    """Remotely start the index on the webserver, for a specific source"""
    
    # connect to webserver via SSH with a SSH key
    host = '136.144.205.98'
    user = 'alexbrandsen'
    port = 22
    
    sshcon = paramiko.SSHClient()  # will create the object
    sshcon.set_missing_host_key_policy(paramiko.AutoAddPolicy()) # no known_hosts error
    pkey = paramiko.RSAKey.from_private_key_file("/home/alex/.ssh/id_rsa")
    sshcon.connect(host, username=user, pkey=pkey) # no passwd needed

    # get datetime, to put in logfile name
    now = datetime.now()
    datetime_string = now.strftime("%Y-%m-%d_%Hh%M")
    
    # run command with nohup and dev/null to not wait for completion, and log to a file
    #(as per https://stackoverflow.com/questions/29142/getting-ssh-to-execute-a-command-in-the-background-on-target-machine)
    command = f"nohup python3 /home/alexbrandsen/upload-json-to-elasticsearch.py {source} >json_import_logs/{source}_{datetime_string}.log 2>&1 </dev/null &"
    stdin, stdout, stderr = sshcon.exec_command(command)

    for line in stdout.readlines():
        print(line)
    for line in stderr.readlines():
        print(line)
    
    # close connection
    sshcon.close()

start_index('dans')